In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from qiskit import BasicAer

from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import algorithm_globals, quantum_instance
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel, BaseKernel
from qiskit import Aer, QuantumCircuit

In [2]:
scaled_df = pd.read_csv('scaled_feature_df.csv')
scaled_df

,mean,variance,skewness,kurtosis,frequency
0,0.522527,0.027656,0.539830,0.078714,0
1,0.695258,0.012871,0.459747,0.143217,0
2,0.810479,0.016648,0.717302,0.215619,0
3,0.696185,0.142455,0.609865,0.047672,0
4,0.625906,0.020161,0.371988,0.154119,0
...,...,...,...,...,...
699,0.535469,0.005455,0.603689,0.076917,3
700,0.541522,0.000272,0.500585,0.126371,3
701,0.578520,0.002821,0.578555,0.123725,3
702,0.493863,0.003348,0.558257,0.073942,3


In [3]:
from sklearn.model_selection import train_test_split

X = scaled_df.drop(columns=["frequency"]).values
y = scaled_df["frequency"].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, stratify=y)

In [ ]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

# Build a feature map
feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")

# Create a quantum kernel
qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [ ]:
# Defining QSVC

qsvc = QSVC(quantum_kernel=qkernel)

In [16]:
# Model Training

import time
start = time.time()

qsvc.fit(X_train, y_train)

print(start - time.time())

-737.422299861908


In [ ]:
eval_df = pd.DataFrame(columns=['predicted_label', 'true_label'])
eval_df['predicted_label'] = qsvc.predict(X_test)
eval_df['true_label'] = y_test

In [28]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, eval_df['predicted_label'].values)
accuracy

0.24822695035460993

In [15]:
# Angle encoding
num_qubits = 4

# Define the number of qubits based on the number of features
num_qubits = len(X_train[0])

# Create a quantum circuit with the specified number of qubits
circuit = QuantumCircuit(num_qubits)

for row in X_train:
    for i, amplitude in enumerate(row):
        # Apply the amplitude encoding to each qubit
        circuit.ry(2 * np.arcsin(amplitude), i)

In [4]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

adhoc_feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement="linear")

adhoc_kernel = FidelityQuantumKernel(feature_map=adhoc_feature_map)

In [5]:
qsvc = QSVC(quantum_kernel=adhoc_kernel)

In [12]:
train_times = []

In [14]:
import time

for num_samples in [20, 40, 80, 160, 320]:
    start = time.time()

    qsvc.fit(X_train[:num_samples, :], y_train[:num_samples])
    
    train_time = time.time() - start
    print(train_time)
    train_times.append(train_time)

0.6654620170593262
2.8072669506073
11.793025970458984
47.2485671043396
195.8482551574707


In [17]:
train_times.append(737.422299861908)

In [19]:
qsvc.save('/Users/sinner/Desktop/trained_QSVC.npz')

In [20]:
eval_df = pd.DataFrame(columns=['predicted_label', 'true_label'])

In [21]:
eval_df['predicted_label'] = qsvc.predict(X_test)

In [22]:
eval_df['true_label'] = y_test

In [23]:
eval_df

,predicted_label,true_label
0,2,3
1,2,1
2,1,1
3,1,3
4,2,3
...,...,...
136,1,0
137,0,3
138,1,0
139,2,3


In [24]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(eval_df['predicted_label'].values, y_test)

In [25]:
accuracy

0.24822695035460993

In [ ]:
accuracy = accuracy_score(eval_df['predicted_label'].values, y_test)

In [31]:
loaded = QSVC.load('/Users/sinner/Desktop/trained_QSVC')

In [32]:
loaded.predict(X_test[10])

array([2, 2, 1, 1, 2, 1, 3, 3, 3, 1])

From https://qiskit.org/documentation/stable/0.24/tutorials/machine_learning/01_qsvm_classification.html

In [37]:
# import matplotlib.pyplot as plt
# import numpy as np

# from qiskit import BasicAer
# from qiskit.circuit.library import ZZFeatureMap
# from qiskit.aqua import QuantumInstance, aqua_globals
# from qiskit.aqua.algorithms import QSVM
# from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

# seed = 10599
# aqua_globals.random_seed = seed

# from qiskit_machine_learning.datasets import ad_hoc_data, sample_ad_hoc_data

# feature_dim = 2
# sample_total, training_input, test_input, class_labels = ad_hoc_data(
#     training_size=20,
#     test_size=10,
#     n=feature_dim,
#     gap=0.3,
#     plot_data=True
# )
# extra_test_data = sample_ad_hoc_data(sample_total, 10, n=feature_dim)
# datapoints, class_to_label = split_dataset_to_data_and_labels(extra_test_data)
# print(class_to_label)

# feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
# qsvm = QSVM(feature_map, training_input, test_input, datapoints[0])

# backend = BasicAer.get_backend('qasm_simulator')
# quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

# result = qsvm.run(quantum_instance)

# print(f'Testing success ratio: {result["testing_accuracy"]}')
# print()
# print('Prediction from datapoints set:')
# print(f'  ground truth: {map_label_to_class_name(datapoints[1], qsvm.label_to_class)}')
# print(f'  prediction:   {result["predicted_classes"]}')
# predicted_labels = result["predicted_labels"]
# print(f'  success rate: {100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)}%')

https://qiskit.org/ecosystem/machine-learning/stubs/qiskit_machine_learning.kernels.BaseKernel.html#qiskit_machine_learning.kernels.BaseKernel

https://qiskit.org/documentation/stable/0.24/tutorials/machine_learning/01_qsvm_classification.html

https://qiskit.org/ecosystem/machine-learning/tutorials/03_quantum_kernel.html

https://qiskit.org/ecosystem/machine-learning/tutorials/08_quantum_kernel_trainer.html

https://www.google.com/search?client=safari&rls=en&q=create+a+quantum+kernel+for+qsvc&ie=UTF-8&oe=UTF-8

https://qiskit.org/documentation/migration_guides/qi_migration.html